In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from sklearn.metrics import roc_auc_score
from sklearn import mixture, preprocessing, datasets

from importlib import reload
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import patches

import torch.utils.data as data_utils

import utils.mahalanobis as models

import utils.dataloaders as dl


In [105]:
reload(models)

<module 'utils.mahalanobis' from '/home/alexm/project/notebooks/gmm-robust/utils/mahalanobis.py'>

In [106]:
device = torch.device('cuda:1')

pre_trained_net = '../deep_Mahalanobis_detector/pre_trained/resnet_cifar10.pth'

train_loader = dl.CIFAR10(train=True)

model = models.ResNet34(num_c=10)
model.load_state_dict(torch.load(pre_trained_net))



IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [107]:
maha = models.Mahalanobis(model.to(device), train_loader, 10, .001, device)
maha.cpu()


 Training Accuracy:(52.00%)



Mahalanobis(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (

In [108]:
torch.save(maha, 'SavedModels/other/mahalanobis/CIFAR10.pth')

In [105]:
data = next(iter(train_loader))[0].to(device)

In [3]:
device = torch.device('cuda:1')

train_loader = dl.CIFAR10(train=True)
data = next(iter(train_loader))[0].to(device)

In [4]:
model = torch.load('SavedModels/other/mahalanobis/CIFAR10.pth').to(device)

In [98]:
maha.to(device)

Mahalanobis(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (

In [109]:
file = 'SavedModels/base/base_CIFAR10_lr0.1_augm_flagTrue_train_typeplain.pth'
pretrained = torch.load(file)

In [100]:
file = 'SavedModels/other/mahalanobis/CIFAR10_base.pth'

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime

import utils.models as models
import utils.plotting as plotting
import utils.dataloaders as dl
import utils.traintest as tt
import utils.adversarial as adv
import utils.eval as ev
import utils.gmm_helpers as gmm_helpers
import model_params as params

from importlib import reload

import utils.mahalanobis as Maha

dataset = 'SVHN'


saving_string = dataset + '_base'
device = torch.device('cpu')

model_params = params.params_dict[dataset](augm_flag=True)


if dataset=='MNIST':
    model = maha.LeNet()
elif dataset=='FMNIST':
    model = maha.ResNet34(10, 1)
elif dataset in ['SVHN', 'CIFAR10']:
    model = maha.ResNet34(10, 3)
elif dataset=='CIFAR100':
    model = maha.ResNet34(100, 3)


model = model.to(device)


param_groups = [{'params':model.parameters(),'lr':model_params.lr, 'weight_decay':5e-4}]
    
if dataset=='MNIST':
    optimizer = optim.Adam(param_groups)
else: 
    optimizer = optim.SGD(param_groups, momentum=0.9)


for epoch in range(100):
    if epoch+1 in [50,75,90]:
        for group in optimizer.param_groups:
            group['lr'] *= .1
 
    trainloss, correct_train = tt.training_dict['plain'](model, device,
                                                          model_params.train_loader,  
                                                          optimizer, epoch, 
                                                          verbose=-1)
    print(str(epoch) + ': \t' + str(correct_train))

model = model.to('cpu')

torch.save(model, 'SavedModels/other/mahalanobis/' + saving_string+ '.pth')


AttributeError: 'Mahalanobis' object has no attribute 'ResNet34'

In [8]:
maha = maha.Mahalanobis(model.to(device), model_params, .001, device)
maha.cpu()

torch.save(maha, 'SavedModels/other/mahalanobis/' + dataset + '.pth')

RuntimeError: Given groups=1, weight of size 32 1 5 5, expected input[2, 3, 32, 32] to have 1 channels, but got 3 channels instead

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime

import utils.models as models
import utils.plotting as plotting
import utils.dataloaders as dl
import utils.traintest as tt
import utils.adversarial as adv
import utils.eval as ev
import utils.gmm_helpers as gmm_helpers
import model_params as params

import utils.mahalanobis as maha

import argparse

dataset = 'SVHN'

saving_string = dataset + '_base'
device = torch.device('cuda:2')

model_params = params.params_dict[dataset](augm_flag=True)


if dataset=='MNIST':
    model = maha.LeNet()
elif dataset=='FMNIST':
    model = maha.ResNet34(10, 1)
elif dataset in ['SVHN', 'CIFAR10']:
    model = maha.ResNet34(10, 3)
elif dataset=='CIFAR100':
    model = maha.ResNet34(100, 3)


model = model.to(device)


param_groups = [{'params':model.parameters(),'lr':model_params.lr, 'weight_decay':5e-4}]
    
if dataset=='MNIST':
    optimizer = optim.Adam(param_groups)
else: 
    optimizer = optim.SGD(param_groups, momentum=0.9)


for epoch in range(100):
    if epoch+1 in [50,75,90]:
        for group in optimizer.param_groups:
            group['lr'] *= .1
 
    trainloss, correct_train = tt.training_dict['plain'](model, device,
                                                          model_params.train_loader,  
                                                          optimizer, epoch, 
                                                          verbose=-1)
    print(str(epoch) + ': \t' + str(correct_train))

model = model.to('cpu')

0: 	0.6856546132110242
1: 	0.9025076101942476
2: 	0.917987359569734
3: 	0.9264643651801193
4: 	0.9285529027942723
5: 	0.9312011138867275
6: 	0.9333852055093711
7: 	0.9342997938763531
8: 	0.9357877062942791
9: 	0.9377943404725828
10: 	0.9376987864640922
11: 	0.9383949656688098
12: 	0.9393778068989994
13: 	0.9399647815225849
14: 	0.9417530065386243
15: 	0.940701912445227
16: 	0.941220634205605
17: 	0.9422171260084361
18: 	0.9429952086490029
19: 	0.9421761742905115
20: 	0.943391075255607
21: 	0.943077112084852
22: 	0.9445513739301363
23: 	0.9437732912895695
24: 	0.944360265913155
25: 	0.9448380359556083
26: 	0.9438688452980603
27: 	0.9443193141952305
28: 	0.9448243853829669
29: 	0.9454932634424014
30: 	0.9452066014169295
31: 	0.9453294565707031
32: 	0.9440053510244755
33: 	0.9459573829122132
34: 	0.9456024680235335
35: 	0.9448653371008914
36: 	0.9464078518093834
37: 	0.9465580081084402
38: 	0.9466399115442893
39: 	0.9451519991263634
40: 	0.9449199393914575
41: 	0.9449608911093821
42: 	0.9

In [2]:
torch.save(model, 'SavedModels/other/mahalanobis/' + saving_string+ '.pth')

In [3]:
final_model = maha.Mahalanobis(model.to(device), model_params, device)
final_model.cpu()

torch.save(final_model, 'SavedModels/other/mahalanobis/' + dataset + '.pth')